In [ ]:
!pip install qiskit

# General Tech Versions/Info
- **Python:** 3.7.10
- **Qiskit**: {'qiskit': '0.24.0',
 'qiskit-aer': '0.7.6',
 'qiskit-aqua': '0.8.2',
 'qiskit-ibmq-provider': '0.12.1',
 'qiskit-ignis': '0.5.2',
 'qiskit-terra': '0.16.4'}
- Run on Google Colab
- Requires 20 qubits + 6 bits

In [2]:
from qiskit import (
    IBMQ,
    QuantumCircuit,
    execute,
    Aer,
    QuantumRegister, 
    ClassicalRegister,
    transpile,
)

from qiskit.ignis.mitigation.measurement import complete_meas_cal, tensored_meas_cal, CompleteMeasFitter,TensoredMeasFitter
from qiskit.test.mock import FakeSydney, FakeMontreal
import numpy as np
import itertools
import math
import random
# import csv
import time
import pandas as pd

In [3]:
# CNOT gates for applying the Steane Code
CNOT_GATES = [[1,0], [1,4], [1,5], [2,0], [2,4], [2,6], [3,4], [3,5],[3,6],[7, 12],[7, 13],[8,7] ,[8, 11],[8, 12], [9,7],[9, 11],[9, 13],[10, 11],[10, 12],[10, 13]]
REV_CNOT_GATES = [[10, 13],[10, 12],[10, 11],[9, 13],[9, 11],[9,7],[8, 12],[8, 11],[8,7],[7, 13],[7, 12],[3,6],[3,5],[3,4],[2,6],[2,4],[2,0],[1,5],[1,4],[1,0]]

#User Defined Values

Please edit these values before running on quantum computer:

In [4]:
# COMPUTATION CONFIGURATIONS
NUM_SHOTS = 8192
NUM_DIFF_PROGRAMS = 10
NUM_ITERATIONS = 1
NUM_RANDOM_ITERATIONS = 10

# SPECIFY NUMBER OF RANDOM SEEDS FOR TRANSPILING QUANTUM CIRCUITS
NUM_SEEDS = 150

# SET PHYSICAL TO VIRTUAL QUBIT MAPPING OF QUANTUM MACHINE
unpermuted_layout = [8,11,13,19,14,20,16,1,2,7,12,6,4,10]
syndrome_layout = [5,9,18,0,3,15]

In [5]:
# SET LOCATION FOR SAVING DATA
from google.colab import drive
drive.mount('/content/drive')

# SET FILENAMES FOR DATA SAVING 
filename_0 = "/content/drive/My Drive/Results/general_info_20Q.txt"
filename_error = "/content/drive/My Drive/Results/error_results_20Q.csv"
filename_mit = "/content/drive/My Drive/Results/mit_results_20Q.csv"
filename_decoded = "/content/drive/My Drive/Results/decoded_results_20Q.csv"

# SET QUANTUM COMPUTER BACKEND OF YOUR CHOICE
fake_mtrl = FakeMontreal()
# BACKEND = AerSimulator.from_backend(fake_mtrl)
BACKEND = Aer.get_backend('qasm_simulator')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Server Class (Main Code)

In [6]:
class Server():
  def __init__(self):
    print('Server initialized')

  def generate_point(self, size=14):
    """
    Generate a random point as password
    
    Parameters:
    size (int): number of qubits in circuit to be encrypted
    
    Returns:
    point (str): the combination of key1 and key2 in binary form
    key1 ([int]): permutation key in integer form
    key2 ([[int], [int]]): one-time pad component of the password, comprised of the x key and z key respectively
    """    
    # get permutation key
    bin_key1, key1 = self.generate_permutation_key()
    # get OTP key
    x_key = np.random.randint(2, size=size)
    z_key = np.random.randint(2, size=size)
    key2 = [x_key.tolist(), z_key.tolist()]
    # combine keys to get point
    a = ''.join(bin_key1)
    b = ''.join(map(str, x_key.tolist()))
    c = ''.join(map(str,z_key.tolist()))
    point = a+b+c
    return point, key1, key2
  
  def generate_permutation_key(self, size=14):
    """
    Generate a random permutation of list(range(size))
    
    Parameters:
    size (int): size of list 
    
    Returns:
    key ([str]): the permuted list in binary form
    dec_key ([int]): the permuted list in decimal form
    """   
    key_dec = list(range(size))
    rng = np.random.default_rng()
    rng.shuffle(key_dec)
    f = '0' + str(math.ceil(math.log2(size))) + 'b'
    # get the permutation in binary form
    key = [format(x, f) for x in key_dec]
    return key, key_dec

  def sample_challenge_point(self, point, size=14): 
    """
    Sample a random point q from the distribution in which with approx.
    probability 0.5, point (the parameter) is sampled, 
    and with approx. probability 0.5, a random point excluding point (the parameter) is uniformly chosen
    
    Parameters:
    point (str): the point that will be sampled with probability 0.5 in the distribution
    size (int): number of qubits that point encrypts for
    
    Returns:
    sample (str): challenge point taken from distribution
    """   
    # generate a random valid point that has a permutation and one-time pad keys
    key1, key_dec = self.generate_permutation_key()
    key1 = "".join(key1)

    key2_dec = random.randint(0, (2**(size*2))-1)
    key2_bin = format(key2_dec,'0'+str(size*2)+'b')

    random_point = str(key1) + str(key2_bin)
    # keep sampling for a random point uniformly until the random_point is not equivalent to point
    while random_point == point:
      key2_dec = random.randint(0, (2**(size*2))-1)
      key2_bin = format(key2_dec,'0'+str(size*2)+'b')
      random_point = str(key1) + str(key2_bin)
    # sample from challenge distribution in which with approx. 50%, random_point is selected, and 50%, point is selected
    sample = np.random.choice([random_point, point])
    return sample

  def protect(self, permuted_cnots, hadamards, x_key, z_key, init_data = [0,0,0,0,0,0,0,'+',0,0,0,0,0,0], size=14):
    """
    Encodes a program 
    
    Parameters:
    permuted_cnots ([[int,int]]): all permuted CNOT gates to be applied
    hadamards ([int]): all hadamard gates to be applied
    x_key ([int]): all pauli-X gates to be applied
    z_key ([int]): all pauli-Z gates to be applied
    init_data (list): initialized qubit states
    size (int): size of the quantum circuit

    Returns:
    circuit (qiskit's QuantumCircuit): encoded program
    """   
    # initialize quantum circuit
    qr = QuantumRegister(size)
    circuit = QuantumCircuit(qr)
    
    # initialize the states of the quantum circuit
    for i in range(size):
      if init_data[i] == '+':
        circuit.h(i)
      elif init_data[i] == 1:
        circuit.x(i)
      elif init_data[i] == '-':
        circuit.x(i)
        circuit.h(i)
    circuit.barrier()
    # apply delegated one-time pad
    for i in range(size):
      if x_key[i] == 1 and init_data[i] == 0:
        circuit.x(i)
      elif z_key[i] == 1 and init_data[i] == '+':
        circuit.z(i)
    circuit.barrier()
    # apply hadamard gates
    for i in hadamards:
      circuit.h(i)  
    circuit.barrier()
    # apply cnot gates
    for cnots in permuted_cnots:
      circuit.cx(cnots[0], cnots[1])
    circuit.barrier()
    return circuit

  def get_syndrome_circuit(self, challenge_input, program, size=14, syndrome_cnots =[[0, 14], [2, 14], [4, 14], [6, 14], [1, 15], [2, 15], [5, 15], [6, 15], [3, 16], [4, 16], [5, 16], [6, 16], [7, 17], [9, 17], [11, 17], [13, 17], [8, 18], [9, 18], [12, 18], [13, 18], [10, 19], [11, 19], [12, 19], [13, 19]]):
    """
    Creates a circuit that detects for single bit and phase flip errors 
    
    Parameters:
    challenge_input (str): point used to decrypt program
    program (qiskit's QuantumCircuit): program for finding error syndromes
    size (int): the number of qubits in the program
    syndrome_cnots ([int,int]): CNOT gates for obtaining error syndromes

    Returns:
    syndrome_circuit (qiskit's QuantumCircuit): program for calculating error syndromes
    """  
    key1, key2 = self.point_to_keys(challenge_input) 
    # initialize quantum circuit
    qr = QuantumRegister(size+int(size/7*3))
    cr = ClassicalRegister(size+int(size/7*3))
    syndrome_circuit = QuantumCircuit(qr, cr)
    # add program to new quantum circuit
    syndrome_circuit.append(program, range(size))
    # apply gates to decrypt the circuit
    for i in range(size,size+int(size/7*3)):
      syndrome_circuit.h(i)

    for gate in syndrome_cnots:
      syndrome_circuit.cx(gate[1], key1.index(gate[0]))

    for i in range(size,size+int(size/7*3)):
      syndrome_circuit.h(i)

    syndrome_circuit.barrier()
    syndrome_circuit.measure(qr,cr)
    return syndrome_circuit


  def get_syndrome_circuit_mit_measures(self, mit_values, challenge_input, program, size=14, syndrome_cnots =[[0, 14], [2, 14], [4, 14], [6, 14], [1, 15], [2, 15], [5, 15], [6, 15], [3, 16], [4, 16], [5, 16], [6, 16], [7, 17], [9, 17], [11, 17], [13, 17], [8, 18], [9, 18], [12, 18], [13, 18], [10, 19], [11, 19], [12, 19], [13, 19]]):
    """
    Creates a circuit that detects bit and phase flip errors but measures only a subset of qubits; 
    Used for tensored error mitigation
    
    Parameters:
    mit_values ([int]): subset of qubits to be measured
    challenge_input (str): point used to decrypt program
    program (qiskit's QuantumCircuit): program for finding error syndromes
    size (int): the number of qubits in the program
    syndrome_cnots ([int,int]): CNOT gates for obtaining error syndromes

    Returns:
    syndrome_program (qiskit's QuantumCircuit): program for calculating error syndromes with partial qubit measurement
    """    
    key1, key2 = self.point_to_keys(challenge_input) 

    qr = QuantumRegister(size+int(size/7*3))
    cr = ClassicalRegister(len(mit_values))
    syndrome_program = QuantumCircuit(qr, cr)
    syndrome_program.append(program, range(size))

    for i in range(size,size+int(size/7*3)):
      syndrome_program.h(i)

    for gate in syndrome_cnots:
      syndrome_program.cx(gate[1], key1.index(gate[0]))

    for i in range(size,size+int(size/7*3)):
      syndrome_program.h(i)

    syndrome_program.barrier()
    for i in range(len(mit_values)):
      syndrome_program.measure(qr[mit_values[i]], cr[i])
    return syndrome_program

  def point_to_keys(self, point, size=14):
    """
    Derives the permutation and one-time pad keys from a point
    
    Parameters:
    point(str): point for deriving keys from
    size (int): number of qubits in program

    Returns:
    circuit (circuit): protected program
    """  
    inc = math.ceil(math.log2(size))
    key1 = [int(point[i:i+inc],2) for i in range(0, len(point[:-size*2]), inc)] 
    key2_x = [int(value) for value in point[-size*2:-size]]
    key2_z = [int(value) for value in point[-size:]]
    return key1, [key2_x, key2_z]

  def permute_classical(self, key1, orig_cnots, hadamards = [1,2,3,8,9,10], size=14):
    """
    Provides the locations of CNOT and Hadamard gates based on a permutated list
    
    Parameters:
    key1 ([int]): permutated list
    orig_cnots ([[int,int]]): the location of unpermuted CNOT gates
    hadamards ([int]): the location of unpermuted Hadamard gates
    size (int): number of qubits in program

    Returns:
    new_cnot_gates ([[int,int]]): permuted CNOT gates
    new_hadamard_gates ([int]): permuted Hadamard gates
    """  
    new_hadamard_gates = [0]*len(hadamards)
    new_cnot_gates = [0]*len(orig_cnots)

    for i in range(len(orig_cnots)):
      new_cnot_gates[i] = [key1.index(orig_cnots[i][0]), key1.index(orig_cnots[i][1])]
    for i in range(len(hadamards)):
      new_hadamard_gates[i] = key1.index(hadamards[i])

    return new_cnot_gates, new_hadamard_gates

  def get_OTP_classical_key(self, key, permutation_key, cnots, hadamards):
    """
    Gets the delegated one-time pad key, where the one-time pad key is delegated to the beginning of the program
    
    Parameters:
    key ([[int],[int]]): the one-time pad key to be delegated 
    permutation_key ([int]): permutation
    cnots ([[int,int]]): all CNOT gates
    hadamards ([int]): all Hadamard gates 

    Returns:
    new_x_key ([int]): delegated Pauli-X gates of one-time pad
    new_z_key ([int]): delegated Pauli-Z gates of one-time pad
    """  
    x_key = key[0]
    z_key = key[1]

    for cnot in cnots:
      a = x_key[cnot[0]]
      b = z_key[cnot[0]]
      c = x_key[cnot[1]]
      d = z_key[cnot[1]]
      x_key[cnot[0]] = a
      z_key[cnot[0]] = b+d
      x_key[cnot[1]] = a+c
      z_key[cnot[1]] = d

    for i in hadamards:
      x_key[i], z_key[i] = z_key[i], x_key[i]

    new_x_key = [i%2 for i in x_key]
    new_z_key = [i%2 for i in z_key]

    return new_x_key, new_z_key


  def undo_circuit(self, point, program, rev_cnots=[[3,6],[3,5],[3,4],[2,6],[2,4],[2,0],[1,5],[1,4],[1,0],[0,6],[0,5],[10, 13],[10, 12],[10, 11],[9, 13],[9, 11],[9, 7],[8, 12],[8, 11],[8, 7],[7, 13],[7, 12]], size=14):
    """
    Applies all the operations in reverse order as to undo the original program
    
    Parameters:
    point (str): the point for encoding the program
    program (qiskit's QuantumCircuit): circuit to be undoed
    rev_cnots ([[int,int]]): the reverse sequence of CNOT gates that were applied in the program
    size (int): number of qubits in program

    Returns:
    undo_circuit (qiskit's QuantumCircuit): the program that has been undoed
    """  
    key1, key2 = self.point_to_keys(point) 
    permuted_cnots, hg = self.permute_classical(key1, rev_cnots)
    qr = QuantumRegister(size)
    cr_trap = ClassicalRegister(size)
    undo_circuit = QuantumCircuit(qr, cr_trap)
    undo_circuit.append(program, range(size))

    for cnot in permuted_cnots:
      undo_circuit.cx(cnot[0], cnot[1])
    
    undo_circuit.barrier()
    for gate in hg:
      undo_circuit.h(gate)
    undo_circuit.barrier()

    undo_circuit.measure(qr, cr_trap)
    return undo_circuit
    
  def reverse_cnots(self, cnots):
    """
    Reverse the order of CNOTs
    
    Parameters:
    cnots ([[int,int]]): original order of cnots

    Returns:
    rev_cnots ([[int,int]]): reversed order of cnots
    """  
    rev_cnots = []
    for i in range(len(cnots)):
      rev_cnots.append(cnots[len(cnots)-i-1])
    return rev_cnots

  def get_random_mit_pattern_single(self, size=20, num_qubits = 10):
    """
    Selected single qubit pattern for tensored error mitigation
    
    Parameters:
    size(int): total number of qubits in the program
    num_qubits(int): number of qubits to be selected

    Returns:
    mit_pattern (list): pattern for tensored error mitigation, comprised of single qubits
    mit_values ([int]): a random subset of all qubits in mit_pattern 
    """  
    mit_vals = random.sample(list(range(size)),num_qubits)
    mit_pattern = [[x] for x in mit_vals]
    return mit_pattern, mit_vals
    
  def get_permuted_cnots(self, permutation_key, cnots):
    """
    Gets the permuted set of CNOTs to be applied for the syndrome programs
    
    Parameters:
    permutation_key([int]): permutation 
    cnots([[int,int]]): CNOT gates to be applied

    Returns:
    new_permuted_cnots ([[int,int]]): permutation of CNOT gates
    """    
    num_aux_qubits = int((len(permutation_key)/7)*3)
    # get the list of auxiliary qubits for obtaining error syndromes
    aux_qubits = list(range(len(permutation_key),len(permutation_key)+num_aux_qubits))
    key = permutation_key + aux_qubits
    new_permuted_cnots = [0]*len(cnots)
    for i in range(len(cnots)):
      new_permuted_cnots[i] = [key.index(cnots[i][0]), key.index(cnots[i][1])]
    return new_permuted_cnots

  def get_random_mit_pattern_all(self, permutation_key, steane_cnots = [[1,0], [1,4], [1,5], [2,0], [2,4], [2,6], [3,4], [3,5],[3,6],[7, 12],[7, 13],[8,7] ,[8, 11],[8, 12], [9,7],[9, 11],[9, 13],[10, 11],[10, 12],[10, 13]], syndrome_cnots = [[14, 0], [14, 2], [14, 4], [14, 6], [15, 1], [15, 2], [15, 5], [15, 6], [16, 3], [16, 4], [16, 5], [16, 6], [17, 7], [17, 9], [17, 11], [17, 13], [18, 8], [18, 9], [18, 12], [18, 13], [19, 10], [19, 11], [19, 12], [19, 13]], size=20, num_qubits = 10):
    """
    Selected single and double qubit patterns for tensored error mitigation
    
    Parameters:
    permutation_key([int]): permutation 
    steane_cnots ([[int,int]]): all cnot gates for the Steane encoding
    syndrome_cnots ([[int,int]]): all cnot gates for calculating the error syndromes
    size(int): total number of qubits in the program
    num_qubits(int): number of qubits to be selected

    Returns:
    mit_pattern (list): pattern for tensored error mitigation, comprised of single and qubit pairs
    mit_values ([int]): a random subset of all qubits in mit_pattern 
    """   
    permuted_steane_cnots = self.get_permuted_cnots(permutation_key, steane_cnots)
    permuted_syndrome_cnots = self.get_permuted_cnots(permutation_key, syndrome_cnots)
    cnots = permuted_steane_cnots  + permuted_syndrome_cnots
    # number of qubit pairs to include in pattern
    num_cnots = random.choice(range(10//2))
    count = 0
    cnot_pairs = []
    cnot_values = []
    while count != num_cnots:
      val = random.choice(range(len(cnots)))
      if cnots[val] not in cnot_pairs:
        if cnots[val][0] not in cnot_values and cnots[val][1] not in cnot_values:
          cnot_pairs.append(cnots[val])
          cnot_values.append(cnots[val][0]) 
          cnot_values.append(cnots[val][1])
          count = count +1 
    
    singles = random.sample(set(list(range(20)))-set(cnot_values),num_qubits-(num_cnots*2))
    s = [[x] for x in singles[:]]
    mit_values = cnot_values + singles
    mit_patterns = cnot_pairs + s
    return mit_patterns,mit_values

  def prepare_meas_filter(self, mit_pattern, backend, num_shots, size=20):
    """
    Prepare a tensored error mitigation measurement filter based on specified mit_pattern
    
    Parameters:
    mit_pattern([int]): pattern used for tensored error mitigation
    backend(qiskit's IBMQBackend): specified backend for preparing measurement filter
    num_shots(int): number of shots for backend
    size(int): number of qubits in program

    Returns:
    meas_filter (qiskit's TensoredMeasFitter.filter): prepared measurement filter
    """  
    qr = QuantumRegister(size) 
    qulayout = range(size)
    meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
    for circ in meas_calibs:
      print(circ.name)
    job = execute(meas_calibs, backend=backend, shots=num_shots)
    cal_results = job.result()
    meas_fitter = TensoredMeasFitter(cal_results, mit_pattern=mit_pattern)
    meas_filter = meas_fitter.filter
    return meas_filter

#Tests

In [7]:
# initiate the server
server = Server()
start = time.time()
fields = ['is_point', 'point_value', 'challenge_point_value', 'key_1', 'key_2', 'challenge_key_1', 'challenge_key_2', 'mit_pattern_single','mit_pattern_all']

Server initialized


In [8]:
results_info = pd.DataFrame(columns=fields)
results_info_decoded = pd.DataFrame(columns=fields)
sp_list= []
sp_mit_single_list =[]
sp_mit_all_list = []
dp_list = []
meas_filter_singles = []
meas_filter_alls = []

In [9]:
def get_programs_for_test(server, challenge_input, program, permutation_key,sp_list, sp_mit_single_list, sp_mit_all_list,dp_list, meas_filter_singles, meas_filter_alls, rev_cnots=REV_CNOT_GATES, backend=BACKEND, num_shots=NUM_SHOTS):
  """
  Prepares circuits for execution

  Parameters:
  server (Server): Server instance
  challenge_input (str): challenge point for testing programs
  program (qiskit's QuantumCicuit): the encoded program for applying tests
  permutation_key ([int]): permutation ordering
  sp_list ([qiskit's QuantumCircuits]): list of prepared syndrome quantum circuits 
  sp_mit_single_list ([qiskit's QuantumCircuits]): list of prepared syndrome quantum circuits with partial measurement and single qubit patterns
  sp_mit_all_list ([qiskit's QuantumCircuits]): list of prepared quantum circuits with syndromes with partial measurement and single and qubit pair patterns
  dp_list ([qiskit's QuantumCircuits]): list of prepared undoed quantum circuits
  meas_filter_singles ([qiskit's TensoredMeasFitter.filter]): list of tensored measurement filters for sp_mit_single_list circuits
  meas_filter_alls ([qiskit's TensoredMeasFitter.filter]): list of tensored measurement filters for sp_mit_all_list circuits
  rev_cnots ([[int,int]]): cnot gates to be applied for undoing the circuit
  backend (qiskit's IBMQBackend): specified backend for preparing measurement filter
  num_shots (int): number of shots for backend

  Returns:
  sp_list ([qiskit's QuantumCircuits]): list of prepared syndrome quantum circuits 
  sp_mit_single_list ([qiskit's QuantumCircuits]): list of prepared syndrome quantum circuits with partial measurement and single qubit patterns
  sp_mit_all_list ([qiskit's QuantumCircuits]): list of prepared quantum circuits with syndromes with partial measurement and single and double qubit patterns
  dp_list ([qiskit's QuantumCircuits]): list of prepared undoed quantum circuits
  meas_filter_singles ([qiskit's TensoredMeasFitter.filter]): list of tensored measurement filters for sp_mit_single_list circuits
  meas_filter_alls ([qiskit's TensoredMeasFitter.filter]): list of tensored measurement filters for sp_mit_all_list circuits
  mit_pattern_s ([[int]]): subset of single qubits used in tensored error mitigation, based on the circuits sp_mit_single_list
  mit_pattern_all (list): subset of single and double qubits used in tensored error mitigation, based on the circuits sp_mit_all_list
  """
  syndrome_program = server.get_syndrome_circuit(challenge_input,program)
  mit_pattern_s, mit_val_s = server.get_random_mit_pattern_single()
  mit_pattern_all, mit_val_all = server.get_random_mit_pattern_all(permutation_key)
  syndrome_program_mit_single = server.get_syndrome_circuit_mit_measures(mit_val_s, challenge_input, program)
  syndrome_program_mit_all = server.get_syndrome_circuit_mit_measures(mit_val_all,challenge_input,program)
  decoded_program = server.undo_circuit(challenge_input, program, rev_cnots=rev_cnots)
  meas_filter_s = server.prepare_meas_filter(mit_pattern_s, backend, num_shots)
  meas_filter_all = server.prepare_meas_filter(mit_pattern_all, backend, num_shots)
  sp_list = sp_list + [syndrome_program]
  sp_mit_single_list =sp_mit_single_list + [syndrome_program_mit_single]
  sp_mit_all_list = sp_mit_all_list + [syndrome_program_mit_all]
  dp_list = dp_list + [decoded_program]
  meas_filter_singles = meas_filter_singles + [meas_filter_s]
  meas_filter_alls = meas_filter_alls + [meas_filter_all]
  return sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all

def prepare_for_test(server, cnots = CNOT_GATES):
  """ 
  Prepare inputs for test

  Parameters:
  server (Server): instance of Server for preparing inputs
  cnots ([[int,int]]): cnot gates to be applied
  
  Returns:
  p (str): point
  k1 ([int]): permutation key
  key2 ([[int],[int]]): one-time pad key
  permuted_cnots([[int,int]]):  cnot gates post permutation
  permuted_hadamards ([int]): hadamard gates post permutation
  x_key ([int]): all delegated pauli-X gates to be applied for one-time pad (key2)
  z_key ([int]):  all delegated pauli-Z gates to be applied for one-time pad (key2)
  data (list): qubits' intial states
  """
  p, k1, k2 = server.generate_point()
  key2 = [k2[0][:], k2[1][:]]
  permuted_cnots, permuted_hadamards = server.permute_classical(k1, cnots)
  rev = server.reverse_cnots(permuted_cnots)
  x_key, z_key = server.get_OTP_classical_key(k2,k1, rev,permuted_hadamards)
  data = [0]*14
  data[k1.index(7)] = '+'
  return p, k1, key2, permuted_cnots, permuted_hadamards, x_key, z_key, data




> ### Test 1.1: Point = Challenge Input Correctness Check

In [10]:
print("_____________PART A: Challenge Input == Point_____________")
for i in range(NUM_DIFF_PROGRAMS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # set challenge_input
  challenge_input = p
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)  
  for k in range(NUM_ITERATIONS): 
    sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
    index = i*NUM_ITERATIONS+k
    results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, mit_pattern_s, mit_pattern_all]
    results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, "-", "-"]


_____________PART A: Challenge Input == Point_____________
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_0000000001
mcalcal_1111111110
mcalcal_1111111111




> ### Test 1.2: Point != Challenge Input, w/ 1 permutation error Correctness Check




In [11]:
print("\n_____________PART B: Challenge Input != Point - one Permutation Error_____________")
for j in range(NUM_DIFF_PROGRAMS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # prepare challenge input
  i = np.random.choice(14,2,False)
  edited_k1 = k1[:]
  edited_k1[i[0]], edited_k1[i[1]] = k1[i[1]], k1[i[0]]
  f = '0' + str(math.ceil(math.log2(14))) + 'b'
  new_key1 = [format(x, f) for x in edited_k1]
  challenge_input = str("".join(new_key1)) + str(p[-28:])
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)

  for k in range(NUM_ITERATIONS): 
    sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
    index = j*NUM_ITERATIONS+k +(NUM_ITERATIONS*NUM_DIFF_PROGRAMS)
    results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, "-","-"]
    results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, mit_pattern_s, mit_pattern_all]



_____________PART B: Challenge Input != Point - one Permutation Error_____________
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_1111111111




> ### Test 1.3: Point != Challenge Input, w/ 1 X error Correctness Check


In [12]:
print("\n_____________PART C: Challenge Input != Point - one X Error_____________")
for j in range(NUM_DIFF_PROGRAMS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # prepare challenge input
  i = np.random.choice(14,1,False)
  index = (i[0]-28)
  challenge_input = str(p[:index]) + str((int(p[index])+1)%2) + str(p[index+1:])
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)
  for k in range(NUM_ITERATIONS): 
    sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
    index = j*NUM_ITERATIONS+k +(NUM_ITERATIONS*NUM_DIFF_PROGRAMS*2)
    results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2,"-","-"]
    results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2,  mit_pattern_s, mit_pattern_all]
print(len(results_info))


_____________PART C: Challenge Input != Point - one X Error_____________
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_0000010101
mcalcal_1111101010
mcalcal_1111111111
3



> ### Test 1.4: Point != Challenge Input, w/ 1 Z error Correctness Check





In [13]:
print("\n_____________PART D: Challenge Input != Point - one Z-Error_____________")
for j in range(NUM_DIFF_PROGRAMS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # prepare challenge input
  i = np.random.choice(14,1,False)
  index = (i[0]-14)
  print(index)
  if i == 13:
    challenge_input = str(p[:index]) + str((int(p[index])+1)%2)
  else:
    challenge_input = str(p[:index]) + str((int(p[index])+1)%2) + str(p[index+1:])
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)
 
  for k in range(NUM_ITERATIONS): 
    sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
    index = j*NUM_ITERATIONS+k +(NUM_ITERATIONS*NUM_DIFF_PROGRAMS*3)
    results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, mit_pattern_s, mit_pattern_all]
    results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2,  "-", "-"]


_____________PART D: Challenge Input != Point - one Z-Error_____________
-1
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_0000010101
mcalcal_1111101010
mcalcal_1111111111



> ### Test 1.5: Point != Challenge Input, w/ 1 X, Z error EACH





In [14]:
print("\n_____________PART E: Challenge Input != Point - one X and Z Error_____________")
for j in range(NUM_DIFF_PROGRAMS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # prepare challenge input
  i = np.sort(np.random.choice(14,2,True))
  x_error_index = i[0] - 28
  z_error_index = i[1] - 14
  if i[1] == 13:
    challenge_input = str(p[:x_error_index]) + str((int(p[x_error_index])+1)%2) + str(p[x_error_index+1: z_error_index])
  else:
    challenge_input = str(p[:x_error_index]) + str((int(p[x_error_index])+1)%2) + str(p[x_error_index+1: z_error_index])+ str((int(p[z_error_index])+1)%2)+ str(p[z_error_index+1:])
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)
  
  for k in range(NUM_ITERATIONS): 
    sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
    index = j*NUM_ITERATIONS+k +(NUM_ITERATIONS*NUM_DIFF_PROGRAMS*4)
    results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, mit_pattern_s, mit_pattern_all]
    results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, "-", "-"]


_____________PART E: Challenge Input != Point - one X and Z Error_____________
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_0000000101
mcalcal_1111111010
mcalcal_1111111111




> ### Test 1.6: Point != Challenge Input, w/ random error Correctness Check



In [15]:
print("\n_____________PART F: Random Challenge Input_____________")
for i in range(NUM_RANDOM_ITERATIONS):
  p, k1, key2, cnots, hadamards, x_key, z_key, data = prepare_for_test(server)
  program = server.protect(cnots, hadamards, x_key, z_key, init_data=data)
  # prepare challenge input
  challenge_input = server.sample_challenge_point(p)
  challenge_key1, challenge_key2 = server.point_to_keys(challenge_input)
  
  sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls, mit_pattern_s, mit_pattern_all = get_programs_for_test(server, challenge_input, program, k1, sp_list, sp_mit_single_list, sp_mit_all_list, dp_list, meas_filter_singles, meas_filter_alls)
  index = (NUM_ITERATIONS*NUM_DIFF_PROGRAMS*5)+i
  results_info.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2, mit_pattern_s, mit_pattern_all]
  results_info_decoded.loc[index] = [str(p) == str(challenge_input), p, challenge_input, k1, key2, challenge_key1, challenge_key2,  "-", "-"]




_____________PART F: Random Challenge Input_____________
mcalcal_0000000000
mcalcal_1111111111
mcalcal_0000000000
mcalcal_1111111111


# Execute Circuits prepared in Tests

### Transpiling Circuits


---



In [16]:
def get_transpiled_circuit_and_depth(circuit_list, backend, init_qubits, opt_level = 2, num_seeds = 1):
  """
  Gets the list of transpiled circuits with the least gate depths based on the random seeds of the specified quantum backend 
  
  Parameters:
  circuit_list ([qiskit's QuantumCircuit]): list of circuits to be transpiled
  backend (qiskit's IBMQBackend): specified quantum computer for transpiling the circuits
  init_qubits ([int]): mapping of virtual to physical qubits
  opt_level (int): the optimization level of the transpiled circuits
  num_seeds (int): the number of random seeds to iterate through
  
  Returns:
  transpiled_list ([qiskit's QuantumCircuit]): transpiled circuits with the least gate depths
  transpiled_depths ([int]): corresponding gate depths of transpiled_list
  """
  transpiled_list = []
  transpiled_depths = []
  for i in range(len(circuit_list)):
    min_circ = transpile(circuit_list[i], backend, initial_layout=init_qubits[i])
    min_depth = min_circ.depth()
    for j in range(num_seeds):
      transpiled_circ = transpile(circuit_list[i], backend, initial_layout=init_qubits[i],optimization_level=opt_level)
      depth = transpiled_circ.depth()
      if depth < min_depth:
        min_depth = depth
        min_circ = transpiled_circ
    transpiled_list.append(min_circ)
    transpiled_depths.append(min_circ.depth())
  return transpiled_list, transpiled_depths

In [17]:
# getting the virtual to physical qubit mapping for all circuits
# mapping is based on the permutation of the circuits and the ideal physical ordering of the quantum computer
init_qubits= []
init_qubits_msg = []
for key1 in results_info.challenge_key_1:
  k1 = key1[:]
  for i in range(len(k1)):
    k1[i]= unpermuted_layout[k1[i]]
  init_qubits_msg.append(k1[:])
  for j in syndrome_layout:
    k1.append(j)
  init_qubits.append(k1)

# getting all the transpiled circuits
transpiled_sp_list, transpiled_sp_depths = get_transpiled_circuit_and_depth(sp_list, BACKEND, init_qubits, num_seeds = NUM_SEEDS)
transpiled_sp_singles_list, transpiled_sp_singles_depths = get_transpiled_circuit_and_depth(sp_mit_single_list, BACKEND, init_qubits, num_seeds =  NUM_SEEDS)
transpiled_sp_all_list, transpiled_sp_all_depths = get_transpiled_circuit_and_depth(sp_mit_all_list, BACKEND, init_qubits, num_seeds =  NUM_SEEDS)
transpiled_sp_msg_list, transpiled_sp_msg_depths = get_transpiled_circuit_and_depth(dp_list, BACKEND, init_qubits_msg, num_seeds =  NUM_SEEDS)


> ### Run Transpiled Circuits on Quantum Computers & Saving to Files

In [18]:
# execute jobs of transpiled error syndrome programs
job = execute(transpiled_sp_list, BACKEND, shots=NUM_SHOTS)
results_sim = job.result()
counts = results_sim.get_counts()
counts = [str(x) for x in counts]

# saving data
results_info.insert(9, "device_counts",counts)
results_info.insert(10, "circuit_depth", transpiled_sp_depths)
results_info.to_csv(filename_error)

mit_counts_singles = []
mit_counts_all = []

# execute jobs of transpiled error syndrome programs (with partial qubit measurement) for error mitigation
job_s = execute(transpiled_sp_singles_list, BACKEND, shots=NUM_SHOTS)
job_all = execute(transpiled_sp_all_list, BACKEND, shots=NUM_SHOTS)
results_sim_s = job_s.result()
counts_s = results_sim_s.get_counts()
results_sim_all = job_all.result()
counts_all = results_sim_all.get_counts()

# get the mitigated counts of the transpiled error syndrome (with partial qubit measurement)
for j in range(NUM_ITERATIONS*NUM_DIFF_PROGRAMS*5+ NUM_RANDOM_ITERATIONS):
  mitigated_counts = meas_filter_singles[j].apply(counts_s[j])
  mit_counts_singles.append(str(mitigated_counts))
  
  mitigated_counts = meas_filter_alls[j].apply(counts_all[j])
  mit_counts_all.append(str(mitigated_counts))

# saving data
results_info.insert(11, "raw_singles", counts_s)
results_info.insert(12,"raw_all", counts_all)
results_info.insert(13, "mitigated_counts_singles",mit_counts_singles)
results_info.insert(14, "singles_circuit_depth", transpiled_sp_singles_depths)
results_info.insert(15, "mitigated_counts_all",mit_counts_all)
results_info.insert(16, "all_circuit_depth", transpiled_sp_all_depths)
results_info.to_csv(filename_mit)

In [19]:
# execute jobs of undoed programs
job = execute(transpiled_sp_msg_list, BACKEND, shots=NUM_SHOTS)
results_sim = job.result()
de_counts = results_sim.get_counts()

# saving data
results_info_decoded.insert(8, "device_counts",de_counts)
results_info_decoded.insert(9, "circuit_depth", transpiled_sp_msg_depths)
results_info_decoded.to_csv(filename_decoded)

In [20]:
# saving some generic data
with open(filename_0, 'w') as writefile:
  x = time.time() - start
  writefile.write("--------------------ELAPSED TIME: \n")
  writefile.write(str(x))
  writefile.write("\n________________________________COUNTS_____________________________________\n")
  writefile.write(str(counts))
  writefile.write("\n________________________________DECODED_COUNTS_____________________________________\n")
  writefile.write(str(de_counts))